# Tests

In [9]:
timeline=tl.stack(
    init(t=-2#,shutter_imaging=0,AOM_imaging=1,trigger_camera=0
        ),
    MOT(t=0.0),
    MOT_detunedGrowth(),
)

In [10]:
tl.display(timeline)

## Testing the difference between `globalRelative=True/False` in `next`

In [23]:
timeline=tl.stack(
    init(t=-2,coil_MOTlower__A=.0,coil_MOTupper__A=.0),
    tl.next(coil_MOTlower__A=-3.0, t=1., time_start=0., context="any"),
    tl.next(coil_MOTupper__A=-4.0, t=1.5, time_start=0.5),
    tl.next(lockbox_MOT__MHz=-5, t=1.5, globalRelative=False)
)

In [24]:
tl.display(timeline)#,variables=["coil_MOTlower__A","coil_MOTupper","lockbox_MOT__MHz"])

In [4]:
timeline=tl.stack(
    init(t=-2#,shutter_imaging=0,AOM_imaging=1,trigger_camera=0
        ),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    tl.wait(1),
    tl.next(
        lockbox_MOT__MHz=0,
        coil_MOTlower__A=-1.0,
        coil_MOTupper__A=-0.98,
        t=.1,
        fargs={"ti": 3},context='sanitize'
    ),
    tl.set(AOM_MOT=1,AOM_repump=1,AOM_imaging=1)
)

In [15]:
prev=tl.previous(timeline,"coil_MOTlower__A")
prev.values[0]=1#["value"]
prev

time                       1
variable    coil_MOTlower__A
value                   -1.0
context                  MOT
Name: 2057, dtype: object

# Imports, connections, devices, constants

In [1]:
import sys
sys.path.append("..")

import importlib.resources as resources
import pathlib as pl

import pandas as pd
import numpy as np
from munch import Munch
from wigner_time import connection as con
from wigner_time import timeline as tl
from wigner_time import adwin as adwin
from wigner_time import constructor as construct
import wigner_time

from copy import deepcopy

%matplotlib qt

In [2]:
connections = con.connection(
    ["shutter_MOT", 1, 11],
    ["shutter_repump", 1, 12],
    ["shutter_OP001", 1, 14],
    ["shutter_OP002", 1, 15],
    ["AOM_MOT", 1, 1],
    ["AOM_repump", 1, 2],
#    ["AOM_OP_aux", 1, 30],  # should be set to 0 always
    ["AOM_OP", 1, 31],
    ["coil_compensationX__A", 4, 7],
    ["coil_compensationY__A", 3, 2],
    ["coil_MOTlower__A", 4, 1],
    ["coil_MOTupper__A", 4, 3],
    ["coil_MOTlowerPlus__A", 4, 2],
    ["coil_MOTupperPlus__A", 4, 4],
    ["lockbox_MOT__MHz", 3, 8],
)

devices = pd.DataFrame(
    columns=["variable", "unit_range", "safety_range"],
    data=[
        ["coil_compensationX__A", (-3, 3), (-3, 3)],
        ["coil_compensationY__A", (-3, 3), (-3, 3)],
        ["coil_MOTlower__A", (-5, 5), (-5, 5)],
        ["coil_MOTupper__A", (-5, 5), (-5, 5)],
        ["coil_MOTlowerPlus__A", (-5, 5), (-5, 5)],
        ["coil_MOTupperPlus__A", (-5, 5), (-5, 5)],
        #["lockbox_MOT__V", (-10, 10)],
        ["lockbox_MOT__MHz", (-200, 200)],
    ],
)

# I dislike global variables but, unfortunately, a reference will probably still need to be available in the same namespace as these functions for convenience.
constants = Munch(
    safety_factor=1.1,
#    factor__VpMHz=0.05,
    lag_MOTshutter=2.48e-3,
    Compensation=Munch(
        Z__A=-0.1,
        Y__A=1.5,
        X__A=0.25,
    ),
    OP=Munch(
        lag_AOM_on=15e-6,
        lag_shutter_on=1.5e-3,
        lag_shutter_off=1.5e-3,
        duration_shutter_on=140e-6,
        duration_shutter_off=600e-6,
    ),
    AI=Munch(
        lag_shutter_on=2.2e-3,
        lag_shutter_off=1.9e-3,
    )
)


# Fundamental sample preparation functions

In [6]:
def init(**kwargs) :
    """
    Creates an experimental timeline for the initialization of every device.
    """
    return tl.create(
        lockbox_MOT__MHz=0.0,
        coil_compensationX__A= constants.Compensation.X__A,
        coil_compensationY__A= constants.Compensation.Y__A,
        coil_MOTlowerPlus__A = -constants.Compensation.Z__A,
        coil_MOTupperPlus__A =  constants.Compensation.Z__A,
        AOM_MOT = 1,
        AOM_repump = 1,
        AOM_OP_aux = 0, # TODO: USB-controlled AOMs should be treated on a higher level
        AOM_OP = 1,
        shutter_MOT = 0,
        shutter_repump = 0,
        shutter_OP001 = 0,
        shutter_OP002 = 1,
        context="ADwin_LowInit",
        **kwargs
    )


def MOT(duration=15, lA=-1., uA=-0.98, **kwargs) :
    return tl.stack(
        tl.set(
#           waitChannel = 0,
            shutter_MOT=1,
            shutter_repump=1,
            coil_MOTlower__A=lA,
            coil_MOTupper__A=uA,
            context='MOT',**kwargs
        ),
        tl.anchor(duration)
    )


def MOT_detunedGrowth(duration=100e-3, durationRamp=10e-3, toMHz=-5, pt=3, **kwargs) :
    return tl.stack(
        tl.next(
            lockbox_MOT__MHz=toMHz,
            t=durationRamp,
            fargs={"ti": pt},context='MOT',**kwargs
        ),
        tl.anchor(duration-durationRamp,#"lockbox_MOT__MHz",
                 )
    )


def molasses(duration=5e-3, durationCoilRamp=9e-4, durationLockboxRamp=1e-3, toMHz=-90, coil_pt=3, lockbox_pt=3, **kwargs) :

    return tl.stack(
        tl.next(
            coil_MOTlower__A=0,
            coil_MOTupper__A=0, # TODO: can these be other than 0 (e.g. for more perfect compensaton?)
            t=durationCoilRamp,
            fargs={"ti": coil_pt},
            context="molasses",
            **kwargs
        ),
        tl.next(
            lockbox_MOT__MHz=toMHz,
            t=durationLockboxRamp,
            fargs={"ti": lockbox_pt}
        ),
        tl.set(
            shutter_MOT=[duration - constants.lag_MOTshutter,0],
            AOM_MOT=[duration,0],
            relative=True),
        tl.wait(1e-5)
    )

def pull_coils(duration, l, u, lp=-constants.Compensation.Z__A, up=constants.Compensation.Z__A, pt = 3, **kwargs) :
    return tl.next(
        coil_MOTlower__A=l,
        coil_MOTupper__A=u,
        coil_MOTlowerPlus__A=lp,
        coil_MOTupperPlus__A=up,
        fargs={"ti": pt},
        t=duration,
        **kwargs,
    )

def magneticTrapping(durationInitial=500e-6, li=-1.8, ui=-1.7, durationStrengthen=3e-3, ls=-4.8, us=-4.7, **kwargs) :
    return tl.stack(
        pull_coils(durationInitial,li,ui,**kwargs),
        pull_coils(durationStrengthen,ls,us)
    )

# Absorption imaging

In [4]:
connections=pd.concat([connections,con.connection(["shutter_imaging", 1, 13],["AOM_imaging", 1, 5],["trigger_camera", 1, 0])])

In [5]:
def trigger_camera(time, exposure, **kwargs) :
    return tl.set(
        ["trigger_camera",[time,1]],
        ["trigger_camera",[time+exposure,0]],
        **kwargs)

def flash_light(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        tl.set(
            ["AOM_imaging",[time,1]],
            ["AOM_imaging",[time+exposure,0]],
            **kwargs),
        tl.set(
            ["shutter_imaging",[time-exposure*(sf-1)-constants.AI.lag_shutter_on,1]],
            ["shutter_imaging",[time+exposure*sf,0]]), # TODO: what to write here exactly?
    )

def expose_camera(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        flash_light(time,exposure,**kwargs),
        trigger_camera(time-exposure*(sf-1)/2, exposure*sf)
    )

def take_image_plus_Bg(time, exposure, delayBg, **kwargs) :
    return tl.stack(
        expose_camera(time, exposure, **kwargs), # taking At image
        trigger_camera(time+delayBg, exposure*constants.safety_factor), # taking Bg_At image
    )    

def imaging_absorption(time, exposure, delayBg, delayLi, exposureBlow=None, **kwargs) :
    """
    Creates an experimental timeline for absorption imaging.
    """
    context = "imaging_absorption"
    return tl.stack(
        tl.set(AOM_imaging=[time-constants.safety_factor*exposure,0],context=context,**kwargs), # initializing the AOM
        take_image_plus_Bg(time, exposure, delayBg), # taking At + Bg_At image
        flash_light(time+5*delayBg,exposureBlow) if exposureBlow is not None else None, # blow out the atoms in between
        take_image_plus_Bg(time+delayLi, exposure, delayBg), # taking Li + Bg_Li image
    )

# Old stuff

In [ ]:
    
    
    def pull_coils(tline, time, duration, l, u, lp=-constants.Compensation.Z__A, up=constants.Compensation.Z__A, pt = 3, context="") :
        return tl.update(
            tl.next(
                ["coil_MOTlower__A", l],
                ["coil_MOTupper__A", u],
                ["coil_MOTlowerPlus__A", lp],
                ["coil_MOTupperPlus__A", up],
                fargs={"ti": pt},
                time_start=time,duration=duration,timeline=tline,context=context,
            ),
        )
    

    def finish(tline, time, duration_ramp, lockbox_initial__V, science_AOM__V, coil_lowerMOT__A, coil_upperMOT__A,ending_parameter_tanh=3) :
        """
        The above but with the new version of next.
        """
        context = "finish"

        return tl.update(
            tl.set(
                timeline=tline,
                context="finish",
                t=duration_total,
                AOM_MOT=1,
                AOM_repump=1,
                AOM_repump__V=5,
                AOM_imaging=1,
                AOM_OP_aux=0,
                AOM_OP=1,
                AOM_science=1,
                AOM_science__V= science_AOM__V,
                AOM_ref=1,
                shutter_MOT=1,
                shutter_repump=1,
                shutter_imaging=0,
                shutter_OP001=0,
                shutter_OP002=1,
                shutter_science=1,
                shutter_transversePump=0,
                shutter_coupling=0,
                trigger_TC__V=0,
                photodiode__V=0,
            ),
            tl.next(
                ["coil_MOTlower__A", coil_lowerMOT__A],
                ["coil_MOTupper__A", coil_upperMOT__A],
                ["coil_MOTlowerPlus__A", -constants.Compensation.Z__A],
                ["coil_MOTupperPlus__A",  constants.Compensation.Z__A],
                ["coil_compensationX__A", constants.Compensation.X__A],
                ["coil_compensationY__A", constants.Compensation.Y__A],
                ["lockbox_MOT__MHz", 0.0],
                # ["lockbox_science__V", params.experiment_CQED.science_light.lockbox_initial__V],
                # ["trigger_RF_modulation__V", 0],
                fargs={"ti": ending_parameter_tanh},
                duration=duration_safely_ramp,
                time_start=time_ramps_before_finish,
                context="before_finish",
            ),
        )

SyntaxError: positional argument follows keyword argument (880452122.py, line 33)

# Thomas’s solution

In [4]:
defaults = Munch()

# NOTE: MOTplus coils are part of the compensation and so should default to the compensation values.
defaults.MOT = Munch(
    lockbox_MOT__V=0.0,
    shutter_MOT=1, # TODO: why the shutter values here?
    shutter_repump=1,
    coil_MOTlower__A=-1.0,
    coil_MOTupper__A=-0.98,
)

defaults.molasses = Munch(
    duration_cooling=5e-3,
    duration_ramp=1e-3,
    shift__MHz=-80,
    fraction_ramp_duration=0.9,
    coil_MOTlower__A=0.0,
    coil_MOTupper__A=0.0,
)

defaults.magnetic = Munch(
    delay=1e-3,
    quadrupole=Munch(duration_ramp=50e-6, coil_MOTlower__A=-1.8, coil_MOTupper__A=-1.7),
    strong=Munch(duration_ramp=3e-3, coil_MOTlower__A=-4.8, coil_MOTupper__A=-4.7),
)

defaults.finish = Munch(set={}, ramp={})
defaults.finish.set = Munch(
    AOM_MOT=1,
    AOM_repump=1,
    AOM_repump__V=5,
    AOM_imaging=1,
    AOM_OP_aux=0,
    AOM_OP=1,
    AOM_science=1,
    AOM_science__V=0.0,
    AOM_ref=1,
    shutter_MOT=1,
    shutter_repump=1,
    shutter_imaging=0,
    shutter_OP001=0,
    shutter_OP002=1,
    shutter_science=1,
    shutter_transversePump=0,
    shutter_coupling=0,
    trigger_TC__V=0,
    photodiode__V=0,
)
defaults.finish.ramp = Munch(
    coil_MOTlower__A=0.0,
    coil_MOTupper__A=0.0,
    coil_MOTlowerPlus__A=-constants.Bfield_compensation_Z__A,
    coil_MOTupperPlus__A=constants.Bfield_compensation_Z__A,
    coil_compensationX__A=constants.Bfield_compensation_X__A,
    coil_compensationY__A=constants.Bfield_compensation_Y__A,
    lockbox_MOT__V=lock_box.to_V(0.0),
)

In [4]:
def init():
    """
    Creates an experimental timeline for the initialization of every relevant variable.
    """
    return tl.create(
        lockbox_MOT__V=0.0,
        coil_compensationX__A=constants.Bfield_compensation_X__A,
        coil_compensationY__A=constants.Bfield_compensation_Y__A,
        coil_MOTlowerPlus__A=-constants.Bfield_compensation_Z__A,
        coil_MOTupperPlus__A=constants.Bfield_compensation_Z__A,
        AOM_MOT=1,
        AOM_repump=1,
        AOM_OP_aux=0,
        AOM_OP=1,
        shutter_MOT=0,
        shutter_repump=0,
        shutter_OP001=0,
        shutter_OP002=1,
        context="ADwin_LowInit",
    )

def MOT(
    detuning__MHz=-5,
    duration=10.0,
    durationFinal=1.0,
    # ===
    time_start=None,
    variables: Munch | None = None,
    variables_default=defaults.MOT,
    timeline=init(),
    context="MOT",
):
    """
    Creates a Magneto-Optical Trap.

    """
    _time_start, _variables = construct.time_and_arguments(
        time_start, variables, variables_default, timeline
    )

    return tl.update(
        tl.create(
            shutter_MOT=_variables.shutter_MOT,
            shutter_repump=_variables.shutter_repump,
            coil_MOTlower__A=_variables.coil_MOTlower__A,
            coil_MOTupper__A=_variables.coil_MOTupper__A,
            # ===
            timeline=timeline,
            context=context,
        ),
        tl.next(
            lockbox_MOT__V=lock_box.to_V(detuning__MHz), t=durationFinal, time_start=duration-durationFinal, context="MOT_grow"
        ),
    )


def molasses(
    duration_cooling=5e-3,
    duration_ramp=1e-3,
    detuning__MHz=-80,
    # ===
    # Specific values above ↑
    # Repeated values below ↓
    # ===
    time_start=None,
    variables: Munch | None = None,
    variables_default=defaults.molasses,
    timeline=MOT(),
    context="molasses",
):
    _time_start, _variables = construct.time_and_arguments(
        time_start, variables, variables_default, timeline
    )

    if duration_ramp >= duration_cooling:
        raise ValueError("duration_ramp should be smaller than duration_cooling!")

    return tl.update(
        tl.create(
            AOM_MOT=[_time_start + duration_cooling, 0],
            shutter_MOT=[_time_start + duration_cooling - constants.lag_MOTshutter, 0],
            timeline=timeline,
            context=context,
        ),
        tl.next(
            coil_MOTlower__A=_variables.coil_MOTlower__A,
            coil_MOTupper__A=_variables.coil_MOTupper__A,
            t=duration_ramp,
        ),
        tl.next(lockbox_MOT__V=lock_box.to_V(detuning__MHz), t=duration_ramp),
    )

In [8]:
init()

,time,variable,value,context
0,0.0,lockbox_MOT__V,0.00,ADwin_LowInit
1,0.0,coil_compensationX__A,0.25,ADwin_LowInit
2,0.0,coil_compensationY__A,1.50,ADwin_LowInit
3,0.0,coil_MOTlowerPlus__A,0.10,ADwin_LowInit
4,0.0,coil_MOTupperPlus__A,-0.10,ADwin_LowInit
5,0.0,AOM_MOT,1.00,ADwin_LowInit
6,0.0,AOM_repump,1.00,ADwin_LowInit
7,0.0,AOM_OP_aux,0.00,ADwin_LowInit
8,0.0,AOM_OP,1.00,ADwin_LowInit
9,0.0,shutter_MOT,0.00,ADwin_LowInit


In [14]:
tl.display(molasses(timeline=MOT(variables={"coil_MOTlower__A":-.5})))

# Params layer

In [164]:
params_default = Munch(
    MOT=Munch(
        duration_grow=15.0,
        duration_final=100e-3, # should be smaller than duration_grow
        shift__MHz=-5.0,
        duration_ramp=10e-3,
        coil_lower__A=-1,
        coil_upper__A=-0.98, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    molasses=Munch(
        duration_cooling=5e-3,
        duration_ramp=1e-3, # should be smaller than duration_cooling
        shift__MHz=-80,
        fraction_ramp_duration=0.9,
        ),
    OP=Munch(
        duration_exposure=80e-6,
        duration_ramp_coils_dipole=500e-6,
        homogeneous_field__A=-0.12,
    ),
    trapping_magnetic=Munch(
        delay_magnetic_trap=1e-3,
        duration_ramp_coils_quadrupole=50e-6,
        duration_ramp_coils_strengthen=3e-3,
        lowerMOT_quadrupole__A=-1.8,
        upperMOT_quadrupole__A=-1.7, # TODO why the asymmetry? this was probably part of magnetic compensation
        lowerMOT_strengthen__A=-4.8,
        upperMOT_strengthen__A=-4.7, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    transport_magnetic=Munch(
        delay=5e-3,
        duration=150e-3,
        lowerMOT__A=-0.3121632,
        upperMOT__A=-4.748368,
        lowerMOTPlus__A=-4.648368,
        upperMOTPlus__A=-4.748368,
        parameter_tanh=2.25,
    ),
    finish=Munch(
        duration_until_finish = 0,
        duration_ramp = 1e-3,
        lockbox_initial__V = 0,
    )
)

In [186]:
def init():
    return preparation_functions.init()

def MOT(tline, time, params,context=""):
    tline = preparation_functions.MOT(tline, time, params.MOT.lowerMOT__A, params.MOT.upperMOT__A)
    tline = preparation_functions.MOT_detunedGrowth(tline, time+params.MOT.duration_grow-params.MOT.duration_final, params.MOT.duration_ramp, 
                                             params.MOT.shift__MHz,pt=3)
    return tline, time+params.MOT.duration_grow

def molasses(tline, time, params):
    tline = preparation_functions.molasses(tline, time, params.molasses.duration_cooling, params.molasses.duration_ramp, params.molasses.duration_ramp, 
                                            params.molasses.shift__MHz)
    return tline, time+params.molasses.duration_cooling

def OP(tline, time, params):
    #def OP(tline, time, duration_exposure, duration_ramp_coils_dipole, duration_shutter_on, homogenous_field__A,coil_parameter_tanh=3):
    tline = preparation_functions.OP(tline, time, params.OP.duration_exposure, params.OP.duration_ramp_coils_dipole, params.OP.homogenous_field__A)
    return tline, time+params.OP.duration_exposure

def magnetic_field(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_quadrupole, params.trapping_magnetic.lowerMOT_quadrupole__A,
                                                    params.trapping_magnetic.upperMOT_quadrupole__A,context="trapping_magnetic")
    time = time+params.trapping_magnetic.duration_ramp_coils_quadrupole
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_strengthen, params.trapping_magnetic.lowerMOT_strengthen__A,
                                             params.trapping_magnetic.upperMOT_strengthen__A,context="trapping_magnetic")
    return tline, time+params.trapping_magnetic.duration_ramp_coils_strengthen

def transport_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.transport_magnetic.duration, params.transport_magnetic.lowerMOT__A,
                                            params.transport_magnetic.upperMOT__A,params.transport_magnetic.lowerMOTPlus__A,params.transport_magnetic.upperMOTPlus__A,
                                            context = "magnetic_transport")
    return tline, time+params.transport_magnetic.duration

def switch_off_trap_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.switch_off_trap_magnetic.duration, 0,0)
    return tline, time+params.switch_off_trap_magnetic.duration

#parameter??
def transport_back_magnetic(tline, time, params):
    tline = preparation_functions.transport_back_magnetic(tline, time, params.transport_back_magnetic.duration, params.transport_back_magnetic.lowerMOT_strengthen__A, params.transport_back_magnetic.upperMOT_strengthen__A)
    return tline, time+params.transport_back_magnetic.duration

def finish(tline, time, params):
    return preparation_functions.finish(tline, time, params.finish.duration_ramp, params.finish.lockbox_initial__V, params.finish.science_AOM__V, params.finish.coil_lowerMOT__A, params.finish.coil_upperMOT__A)

In [190]:
def prepare_atoms(params):
    tline = init()
    tline,timeMOT = MOT(tline, 0, params)
    tline,timemolasses = molasses( *MOT(tline, 0, params), params)
    tline,timeOP = OP(tline, timemolasses, params)
    tline,timeTrapping = magnetic_field(tline, timeOP+params.trapping_magnetic.delay_magnetic_trap, params)
    tline = finish(tline, timeTrapping, params)
    return tline

In [191]:
params=deepcopy(params_default)
params.finish.duration_until_finish = 500e-3

tline = prepare_atoms(params)

In [192]:
%matplotlib qt
tl.display(tline,variables=['lockbox_MOT__V','AOM_MOT','AOM_repump','shutter_repump','AOM_repump__V'],xlim=[15,15.05])

0 Axes(0.125,0.747241;0.775x0.132759) lockbox_MOT__V
1 Axes(0.125,0.587931;0.775x0.132759) AOM_MOT
2 Axes(0.125,0.428621;0.775x0.132759) AOM_repump
3 Axes(0.125,0.26931;0.775x0.132759) shutter_repump
4 Axes(0.125,0.11;0.775x0.132759) AOM_repump__V
